In [ ]:
import pandas as pd
import geojson

In [ ]:
#be careful running cell, uses a lot of memory
geojsonlocation = "./Data/Geojson/ConDistricts"
geoyears = [2012,2016,2020]
for gyear in geoyears:
    geofile = geojsonlocation+str(gyear)+".geojson"
    with open(geofile, 'r') as f:
        geojson_data = geojson.load(f)
        print(geojson_data)